In [1]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
import numpy as np
import folium as f
import folium.plugins as plugins
from folium.features import CustomIcon
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
import os

In [2]:
locator = Nominatim(user_agent='myGeocoder')
user_location = locator.geocode(input('Digite o endeço da sua localização atual. Não use abreviações! Ex: em vez de usar "Av. Sgt Hermínio Sampaio - Fortaleza", use "Avenida Sargento Hermínio Sampaio - Fortaleza". \n'))

Digite o endeço da sua localização atual. Não use abreviações! Ex: em vez de usar "Av. Sgt Hermínio Sampaio - Fortaleza", use "Avenida Sargento Hermínio Sampaio - Fortaleza". 
Rua Bartolomeu Dias, 465, Messejana


In [3]:
user_location

Location(Rua Bartolomeu Dias, Messejana, Fortaleza, Microrregião de Fortaleza, Região Geográfica Intermediária de Fortaleza, Ceará, Região Nordeste, 60841240, Brasil, (-3.8257491, -38.4904167, 0.0))

In [4]:
print("Latitude = {}, Longitude = {}".format(user_location.latitude, user_location.longitude))

Latitude = -3.8257491, Longitude = -38.4904167


In [5]:
df = pd.DataFrame({'Place':
                      ['Oficina da Bike',
                       'Ciclo Peças O Irmão',
                       'Casa do Ciclista',
                       'Ciclopeças O Amigão',
                       'NT Bike'],
                    'Address':
                     ['Rua Guilherme Pessoa 577, Messejana, Fortaleza',
                      'Rua Doutor Joaquim Bento 345, Messejana, Fortaleza',
                      'Rua Doutor Pergentino Maia 552, Messejana, Fortaleza',
                      'Rua Coronel Francisco Pereira 300, Messejana, Fortaleza',
                      'Avenida Odilon Guimarães 3708, Lagoa Redonda, Fortaleza']
})

In [6]:
df.head()

,Place,Address
0,Oficina da Bike,"Rua Guilherme Pessoa 577, Messejana, Fortaleza"
1,Ciclo Peças O Irmão,"Rua Doutor Joaquim Bento 345, Messejana, Forta..."
2,Casa do Ciclista,"Rua Doutor Pergentino Maia 552, Messejana, For..."
3,Ciclopeças O Amigão,"Rua Coronel Francisco Pereira 300, Messejana, ..."
4,NT Bike,"Avenida Odilon Guimarães 3708, Lagoa Redonda, ..."


In [7]:
# convenient function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# create location column
df['location'] = df['Address'].apply(geocode)
# create Longitude, Latitude and altitude from location column (returns list)
df['point'] = df['location'].apply(lambda loc: list(loc.point) if loc else None)
# split point column into Latitude, Longitude and altitude columns
df[['Latitude', 'Longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
# remove location, point and altitude columns
df = df.drop(['location','point','altitude'], axis=1)

In [9]:
df.head()

,Place,Address,Latitude,Longitude
0,Oficina da Bike,"Rua Guilherme Pessoa 577, Messejana, Fortaleza",-3.827106,-38.487960
1,Ciclo Peças O Irmão,"Rua Doutor Joaquim Bento 345, Messejana, Forta...",-3.828313,-38.483291
2,Casa do Ciclista,"Rua Doutor Pergentino Maia 552, Messejana, For...",-3.830711,-38.493905
3,Ciclopeças O Amigão,"Rua Coronel Francisco Pereira 300, Messejana, ...",-3.830873,-38.492815
4,NT Bike,"Avenida Odilon Guimarães 3708, Lagoa Redonda, ...",-3.820065,-38.475501


In [10]:
def euclidean_dist(data, p):
    """Function to calculate the Euclidean Distance"""
    x = (data - p)**2
    return np.sqrt(np.sum(x, axis=1))

In [11]:
array = np.array(list(zip(df['Latitude'].values, df['Longitude'].values)))

In [12]:
my_location = np.array([[user_location.latitude, user_location.longitude]])

In [13]:
df['Euc_Dist'] = euclidean_dist(array, my_location)
df = df.sort_values(by='Euc_Dist')
df = df.reset_index(drop=True)
df

,Place,Address,Latitude,Longitude,Euc_Dist
0,Oficina da Bike,"Rua Guilherme Pessoa 577, Messejana, Fortaleza",-3.827106,-38.487960,0.002807
1,Ciclopeças O Amigão,"Rua Coronel Francisco Pereira 300, Messejana, ...",-3.830873,-38.492815,0.005658
2,Casa do Ciclista,"Rua Doutor Pergentino Maia 552, Messejana, For...",-3.830711,-38.493905,0.006065
3,Ciclo Peças O Irmão,"Rua Doutor Joaquim Bento 345, Messejana, Forta...",-3.828313,-38.483291,0.007573
4,NT Bike,"Avenida Odilon Guimarães 3708, Lagoa Redonda, ...",-3.820065,-38.475501,0.015962


In [14]:
closer_idx = df['Euc_Dist'].idxmin()
closer = df.loc[closer_idx,'Address']
print(closer)

Rua Guilherme Pessoa 577, Messejana, Fortaleza


In [15]:
#display the place names when the user click on the icons
popups = ['{}'.format(place) for place in df['Place']]

#set theme, zoom e initial map location
f_map = f.Map(location=[user_location.latitude, user_location.longitude],
                        zoom_start=12,
                        tiles='cartodbpositron') #'CartoDB dark_matter' (dark theme)
#mark the user location
f.Marker(
    [user_location.latitude, user_location.longitude],
    popup='You are here',
    icon=f.Icon(color='blue', icon='street-view', prefix='fa')
).add_to(f_map)

#groups nearby points (bike shops) into a cluster
marker_cluster = MarkerCluster().add_to(f_map)
for point in range(len(array)):
    f.Marker(array[point], icon=f.Icon(color='red', icon='bicycle', prefix='fa'), 
                  popup=popups[point]).add_to(marker_cluster)
    
#make a line between user location and the nearest bike shop
line = [[user_location.latitude, user_location.longitude],[df.loc[closer_idx,'Latitude'],df.loc[closer_idx,'Longitude']]]

line_mark =  f.PolyLine(line, color="red", weight=2, opacity=1).add_to(f_map)

attr = {'fill': 'black', 'font-size': '20'}

plugins.PolyLineTextPath(
    line_mark,
    '\u27A1\uFE0F', #\U0001f6b4 (bike) #\u25BA #\U0001f463 #\u27A1\uFE0F (arrow)
    repeat = True,
    offset = 7,
    attributes = attr
).add_to(f_map)

formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(f_map)

marker_cluster.add_to(f_map)

f.LayerControl().add_to(f_map)
f_map